# CSV to RDF

* author: Christopher Pollin
* date: 2021


first we have to install the library [rdflib](https://rdflib.readthedocs.io/en/stable/) and import it next to [pandas](https://pandas.pydata.org/).

In [293]:
!pip install rdflib
from rdflib import Graph, Literal, Namespace, URIRef
from rdflib.namespace import DCTERMS, RDF, RDFS, SKOS, XSD
import pandas as pd
import urllib

You should consider upgrading via the 'c:\python38\python.exe -m pip install --upgrade pip' command.


In [294]:
# result_graph contains the rdf graph we want to generate and serialize in a new output file. we predefine alle needed namespaces and add them to result_graph.
result_graph = Graph()

# namespaces, @prefix in rdf
VOID = Namespace("http://rdfs.org/ns/void#")
DCTERMS = Namespace("http://purl.org/dc/terms/")
DC = Namespace("http://purl.org/dc/elements/1.1/")
OT = Namespace("http://wallscope.co.uk/resource/olympics/team/")
DBO = Namespace("http://dbpedia.org/ontology/")
DBP = Namespace("http://dbpedia.org/property/")
OA = Namespace("http://wallscope.co.uk/resource/olympics/athlete/")
OC = Namespace("http://wallscope.co.uk/resource/olympics/city/")
O = Namespace("http://wallscope.co.uk/resource/olympics/")
FOAF = Namespace("http://xmlns.com/foaf/0.1/")
RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")
XSD = Namespace("http://www.w3.org/2001/XMLSchema#")
SKOS = Namespace("http://www.w3.org/2004/02/skos/core#")

result_graph.bind("void", VOID)
result_graph.bind("dcterms", DCTERMS)
result_graph.bind("dc", DC)
result_graph.bind("ot", OT)
result_graph.bind("dbo", DBO)
result_graph.bind("dbp", DBP)
result_graph.bind("oa", OA)
result_graph.bind("oc", OC)
result_graph.bind("o", O)
result_graph.bind("foaf", FOAF)
result_graph.bind("rdfs", RDFS)
result_graph.bind("xsd", XSD)
result_graph.bind("skos", SKOS)

BASE_URL = "http://wallscope.co.uk/resource/olympics/athlete/"

loading the data into a new panda dataframe. upload the `athlete_events_smaller.csv` into the *data* folder. the csv contains 2500 athletes for each multiple rows can exist. the header of the CSV looks like the following;

"ID","Name","Sex","Age","Height","Weight","Team","NOC","Games","Year","Season","City","Sport","Event","Medal"

**YOU HAVE TO REUPLOAD THE CSV INPUT INTO THE CORRECT FOLDER "content" !!!**

In [295]:
# load csv and print it
df = pd.read_csv("data/athlete_events_smaller.csv", encoding="utf8") 
#print(df)

#### `ot:Netherlands a dbo:SportsTeam` 

get all distinct teams and create `ot:Netherlands a dbo:SportsTeam` with rdfs:label

* add @en to data literal via `lang` param 

In [296]:
def normalizeAndEncodeString(string):
  string = str(string).replace(" ", "")
  string = urllib.parse.quote(string)
  return string

In [297]:
for team in df['Team'].unique():
    try:
        team_uri = URIRef("http://wallscope.co.uk/resource/olympics/team/" + normalizeAndEncodeString(team))
        result_graph.add((team_uri, RDF.type, DBO.SportsTeam))
        result_graph.add((team_uri , RDFS.label, Literal(team, lang='en') ))
    except:
        print("Log: failed to create a team_uri")    


#### `oc:SquawValley a dbo:City` 

* get all distinct cities and create `oc:SquawValley a dbo:City`  with rdfs:label
* `str(city).replace(" ", ""))` is necessary as whitespace in "Squaw Valley" leads to an invalid URI. so you have to be careful what you take to build the URI. 

In [298]:
for city in df['City'].unique():
    city_uri = URIRef("http://wallscope.co.uk/resource/olympics/city/" + normalizeAndEncodeString(city) )
    result_graph.add((city_uri, RDF.type, DBO.City))
    # add @en to data literal via lang param
    result_graph.add((city_uri , RDFS.label, Literal(city, lang='en') ))


In [299]:
# o:SpeedSkating a dbp:Sport .
for sport in df['Sport'].unique():
    sport_uri = URIRef("http://wallscope.co.uk/resource/olympics/" + normalizeAndEncodeString(sport) )
    result_graph.add((sport_uri, RDF.type, DBO.Sport))
# o:Summer a dbo:TimePeriod .
for season in df['Season'].unique():
    season_uri = URIRef("http://wallscope.co.uk/resource/olympics/" + normalizeAndEncodeString(season) )
    result_graph.add((season_uri, RDF.type, DBO.TimePeriod))
    result_graph.add((season_uri , RDFS.label, Literal(season, lang='en') ))
# o:SpeedSkatingWomen500metres a o:Discipline .
for discipline in df['Event'].unique():
    # just for having a valid url; maybe not the best idea ;)
    discipline_uri = URIRef("http://wallscope.co.uk/resource/olympics/" + normalizeAndEncodeString(discipline) )
    result_graph.add((discipline_uri, RDF.type, O.Discipline))
    result_graph.add((discipline_uri , RDFS.label, Literal(discipline, lang='en') ))

# add Male and Female as skos:Concept
female = URIRef("http://wallscope.co.uk/resource/olympics/F")
male = URIRef("http://wallscope.co.uk/resource/olympics/M")
result_graph.add(( female , RDF.type, SKOS.Concept))
result_graph.add(( female , RDFS.label, Literal("female", lang='en') ))
result_graph.add(( female , RDFS.label, Literal("weiblich", lang='de') ))
result_graph.add(( male, RDF.type, SKOS.Concept))
result_graph.add(( male , RDFS.label, Literal("male", lang='en') ))
result_graph.add(( male , RDFS.label, Literal("männlich", lang='de') ))

In [300]:

# create <ol:Athlete rdf:about="https://gams.uni-graz.at/olympia.1#9792"/>
# groupy by value in the ID-column
df_group_by_id = df.groupby('ID')
# iteration over all groups
for ID, df_group in df_group_by_id:

  # we will take <http://wallscope.co.uk/resource/olympics/athlete/24> instead of oa:NilsEgilAaness 
  athlete_uri = URIRef(BASE_URL + str(ID))
  result_graph.add((athlete_uri, RDF.type, FOAF.Person))

  # iteration over all elements inside the group
  for row_index, row in df_group.iterrows():
    # foaf:name, rdfs:label
    if(row["Name"]):
      result_graph.add(( athlete_uri, RDFS.label, Literal(row["Name"]) ))
      result_graph.add(( athlete_uri, FOAF.name, Literal(row["Name"]) ))  
    # Age - foaf:age 17, Weight - dbp:weight 65, Height- dbp:height 169; only integer
    if(row["Age"]):
      # integer is default datatype for number; but with datatype=XSD.integer or datatype=XSD.float you can add this to the literal
      result_graph.add(( athlete_uri, FOAF.age, Literal(row["Age"] , datatype=XSD.integer) ))
    if(row["Height"].is_integer()):
      result_graph.add(( athlete_uri, DBP.height, Literal(row["Height"] ) ))
    if(row["Weight"].is_integer()):
      result_graph.add(( athlete_uri, DBP.weight, Literal(row["Weight"] ) ))

    # Sex - foaf:gender o:F
    if(row["Sex"] == "F"):
      result_graph.add((athlete_uri, FOAF.gender, URIRef( female ) ))
    if(row["Sex"] == "M"):
      result_graph.add((athlete_uri, FOAF.gender, URIRef( male ) ))

    # Team - dbp:team ot:Netherlands
    if(row["Team"]):
      result_graph.add(( athlete_uri, DBP.team, URIRef("http://wallscope.co.uk/resource/olympics/team/" + normalizeAndEncodeString(row["Team"]) )))

    # Game
    if(row["Games"]):
      game_uri = URIRef("http://wallscope.co.uk/resource/olympics/" + str(row["Games"]).replace(" ", ""))
      result_graph.add((game_uri, RDF.type, DBO.Olympics))
      # add @en to data literal via lang param
      result_graph.add((game_uri , RDFS.label, Literal( row["Games"], lang='en' ) ))
      result_graph.add((game_uri , O.city, URIRef("http://wallscope.co.uk/resource/olympics/city/" + str(row["City"]).replace(" ", "")) ))
      result_graph.add((game_uri , O.season, URIRef("http://wallscope.co.uk/resource/olympics/" + str(row["Season"]).replace(" ", "")) ))
 
    # o:Discipline o:sport o:Luge.
    discipline_string = urllib.parse.quote(str(row["Event"]).replace(" ", ""))
    discipline_uri = URIRef("http://wallscope.co.uk/resource/olympics/" + discipline_string )
    result_graph.add((discipline_uri , O.sport, URIRef("http://wallscope.co.uk/resource/olympics/" + str(row["Sport"]).replace(" ", "")) ))

    # Result
    result_uri = URIRef("http://wallscope.co.uk/resource/olympics/" + "result." + str(ID))
    result_graph.add((result_uri, RDF.type, O.Result))
    result_graph.add((result_uri, O.athlete, athlete_uri)) 
    result_graph.add((result_uri, O.discipline, discipline_uri)) 
    result_graph.add((result_uri, O.game, game_uri)) 

SyntaxError: invalid syntax (<ipython-input-300-a9ba59e64900>, line 20)

#### Create a new file

* format="xml" - creates plain xml/rdf
* format="pretty-xml" - abbreviated RDF/XML syntax 
* format="turtle"

In [192]:
result_graph.serialize(destination = "olympia_output.ttl", format="turtle")